In [1]:
import numpy as np
import pandas as pd
import os
from joblib import Parallel,delayed

In [ ]:
# testing module

import matplotlib.pyplot as plt

code="601211.SH"
date=20191023
pre_date=20191022

close_seq=np.array(data.loc[(code,date)]["close"])
pre_close_seq=np.array(data.loc[(code,pre_date)]["close"][-60:])
pre_close=data.loc[(code,date)]["pre_close"][0]
today_open=data.loc[(code,date)]["open"][0]

plt.plot(pre_close_seq)
plt.grid()
plt.show()

plt.plot(close_seq)
plt.grid()
plt.show()

print(cross_count(pre_close_seq,close_seq,pre_close))


In [1]:
#!/home/ywang/env/dev/bin
#-*-coding:utf-8-*-
# written by wangyonghao

# the api: get_daily_cross_data(date)

# input:  date: current date ( "20190102" for example)

# output to file: the csv file of the cross_grid_count for each stock

import numpy as np
import pandas as pd
import os
from joblib import Parallel, delayed
import tushare as ts
from data_tools.api import *
from utilscht.Data import *
import pymysql
import datetime
import logging

def get_cross_grid_info(price_now, ref_price,grids):
    if price_now > ref_price:
        cross_grids = [i for i in grids if i > ref_price and i <= price_now]
        if len(cross_grids) > 0:
            grids_num = len(cross_grids)
            direction = 1
            last_grid = cross_grids[-1]

    else:
        cross_grids = [i for i in grids if i > price_now and i <= ref_price]
        if len(cross_grids) > 0:
            grids_num = len(cross_grids)
            direction = -1
            last_grid = cross_grids[0]

    if len(cross_grids) > 0:
        return (grids_num, direction, last_grid)
    else:
        return -1


# 输入前一天最后 60min 的close序列、今天的全部close序列、前一天的收盘价、今天的开盘价
# ，返回今天穿越网格线的次数（一段时间内重复穿过同一条线只计算一次）
def get_cross_count(pre_close_seq, close_seq,grid_unit=0.1):
    total_seq=np.concatenate([pre_close_seq,close_seq])
    if (np.nanmin(total_seq)+np.nanmax(total_seq))/2>=10:
        grid_unit=(np.nanmin(total_seq)+np.nanmax(total_seq))/2*0.01
    else:
        grid_unit = 0.1
    grid_num = int((np.nanmax(total_seq)-np.nanmin(total_seq))/grid_unit+3)
    grids = np.linspace(np.nanmin(total_seq)-grid_unit,np.nanmax(total_seq)+grid_unit,grid_num)
    
    count = 0
    ref_price = 0
    pre_cross_dire = 1

    # get the last corss_line for yesterday
    for i in reversed(range(len(pre_close_seq))):
        price_pre = pre_close_seq[i - 1]
        price_now = pre_close_seq[i]
        cross_grid_info = get_cross_grid_info(price_now, price_pre,grids)

        if cross_grid_info != -1:
            ref_price = cross_grid_info[2]
            pre_cross_dire = cross_grid_info[1]
            break

    # count the cross times for today
    close_seq = np.concatenate([np.array([pre_close_seq[-1]]),close_seq])
    for i in range(len(close_seq) - 1):
        price_pre = close_seq[i]
        price_now = close_seq[i + 1]

        cross_grid_info = get_cross_grid_info(price_now, price_pre,grids)

        if cross_grid_info == -1:
            continue

        # judge the status of "now" and "pre"
        grids_num = cross_grid_info[0]
        cross_dire = cross_grid_info[1]
        last_grid = cross_grid_info[2]

        if last_grid != ref_price:
            if cross_dire * pre_cross_dire == 1:
                count = count + grids_num
            else:
                count = count + grids_num - 1

        ref_price = last_grid
        pre_cross_dire = cross_dire

    return count


# current function to get 1min_freq data
def get_stk_data(sid, date):
    
    try:
        DTYPE_STK_BAR_1MIN = [
            ('wcode', 'S16'),
            ('date', 'i4'),
            ('time', 'i4'),
            ('open', 'i4'),
            ('high', 'i4'),
            ('low', 'i4'),
            ('close', 'i4'),
            ('volume', 'i8'),
            ('value', 'i8'),
            ('num_trades', 'i4'),
            ('unused', 'i4'),
        ]



        bar=np.memmap(r"/dat/raw/wind/stk_bar_1min/{}/{}.dat".format(date,sid),dtype=DTYPE_STK_BAR_1MIN,mode="r")
        df = pd.DataFrame(bar)[["wcode","date","time","close"]]
        df["sid"] = df["wcode"].apply(lambda x:str(x)[2:-1])
        df["DataDate"] = df["date"].apply(str)
        df["close"] = df["close"]/10000
        df["pre_close"] = get_stk_bar(sid, freq="1d", start=date, end=date, fields=["pre_close"]).values[0][0]
        df = df.drop(["wcode","date","time"],axis=1)
        df["ticktime"] = df.index
        return df
    except Exception as e:
        try:
            stock_data = get_stk_bar(sid, freq="1m", start=date, end=date, fields=["open", "high", "low", "close"])
            stock_data = stock_data.reset_index().rename(columns={"index": "datetime"})
            stock_data["ticktime"] = stock_data["datetime"].index
            stock_data["DataDate"] = stock_data["datetime"].apply(lambda x:str(x.date()))
            del stock_data["datetime"]
            stock_data["pre_close"] = get_stk_bar(sid, freq="1d", start=date, end=date, fields=["pre_close"]).values[0][0]
            stock_data["sid"] = sid
            return stock_data
        except:
            logging.warning("Data Lost for {} on {}".format(sid,date))


def get_daily_data(date):
    df=query_table("DailyBar",start_date=date,end_date=date,fields=["tradable"])
    stk_ls=list(df["sid"][df["tradable"]==1])

    results = Parallel(n_jobs=16, verbose=5, backend="loky", batch_size='auto') \
        (delayed(get_stk_data)(sid, date) for sid in stk_ls)
    data = pd.concat(results)

    return data.sort_values(["sid", "ticktime"])


def get_count_parallel(df):
    sid = df["sid"].drop_duplicates()
    temp_df = pd.DataFrame(index=sid, columns=["cross_count"])

    close_seq = np.array(df["close"])
    pre_close_seq = np.array(df["yesterday_close"].iloc[-60:])
    pre_close = df["pre_close"].iloc[0]
    if pre_close!=pre_close_seq[-1]:
        pre_close_seq = pre_close_seq* pre_close/pre_close_seq[-1]

    count = get_cross_count(pre_close_seq, close_seq)

    temp_df.iloc[0, 0] = count
    return temp_df


def apply_parallel(df_grouped, func, n_jobs=16, backend='loky', as_index=False, **kwargs):
    """
    This is for parallel between grouped generated by pd.DataFrame.groupby
    :param df_grouped:
    :param func:
    :param n_jobs:
    :param backend:
    :param kwargs:
    :return:
    """

    names = []
    groups = []
    for name, group in df_grouped:
        names.append(name)
        groups.append(group)

    results = Parallel(n_jobs=n_jobs, verbose=5, backend=backend, batch_size='auto') \
        (delayed(func)(group, **kwargs) for group in groups)

    return pd.concat(results, keys=names if as_index else None)



def get_daily_cross_data(date):
    pre_date = get_previous_trade_date(date)

    data = get_daily_data(date)
    yesterday_data = get_daily_data(pre_date)
    yesterday_data.rename(columns={"close": "yesterday_close"}, inplace=True)
    data = pd.merge(data, yesterday_data[["sid", "ticktime", "yesterday_close"]], how="left", on=["sid", "ticktime"])

    groups = data.groupby("sid")
    cross_count_df = apply_parallel(groups, get_count_parallel)
    cross_count_df = cross_count_df.reset_index()
    cross_count_df["sid"] = cross_count_df["sid"].apply(lambda x:x[0:9])
    cross_count_df.to_csv(r"/share/intern_share/stk_bundle_v2/crosscount_{}.csv".format(date), index=False)

In [2]:
date_ls=get_trade_dates(20180101,20190102)
for date in date_ls[1:]:
    get_daily_cross_data(str(date))
    print(date)
    

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.3s
[Parallel(n_jobs=16)]: Done 148 tasks      | elapsed:    2.0s
[Parallel(n_jobs=16)]: Done 624 tasks      | elapsed:    4.6s
[Parallel(n_jobs=16)]: Done 1272 tasks      | elapsed:    7.0s
[Parallel(n_jobs=16)]: Done 2064 tasks      | elapsed:   10.9s
[Parallel(n_jobs=16)]: Done 3000 tasks      | elapsed:   14.4s
[Parallel(n_jobs=16)]: Done 3227 out of 3258 | elapsed:   15.1s remaining:    0.1s
[Parallel(n_jobs=16)]: Done 3258 out of 3258 | elapsed:   15.2s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.4s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    2.0s
[Parallel(n_jobs=16)]: Done 336 tasks      | elapsed:    3.0s
[Parallel(n_jobs=16)]: Done 664 tasks      | elapsed:    4.3s
[Parallel(n_jobs=16)]: Done 1456 tasks      | elapsed:    6.8s
[Par

20180103


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.3s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    1.9s
[Parallel(n_jobs=16)]: Done 618 tasks      | elapsed:    9.6s
[Parallel(n_jobs=16)]: Done 1016 tasks      | elapsed:   11.0s
[Parallel(n_jobs=16)]: Done 1808 tasks      | elapsed:   13.6s
[Parallel(n_jobs=16)]: Done 2324 tasks      | elapsed:   25.4s
[Parallel(n_jobs=16)]: Done 3225 out of 3256 | elapsed:   28.4s remaining:    0.3s
[Parallel(n_jobs=16)]: Done 3256 out of 3256 | elapsed:   28.5s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.4s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    2.0s
[Parallel(n_jobs=16)]: Done 624 tasks      | elapsed:    5.6s
[Parallel(n_jobs=16)]: Done 914 tasks      | elapsed:   20.9s
[Parallel(n_jobs=16)]: Done 1600 tasks      | elapsed:   23.2s
[Par

20180104


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.3s
[Parallel(n_jobs=16)]: Done 216 tasks      | elapsed:    2.0s
[Parallel(n_jobs=16)]: Done 624 tasks      | elapsed:   11.7s
[Parallel(n_jobs=16)]: Done 968 tasks      | elapsed:   13.1s
[Parallel(n_jobs=16)]: Done 1760 tasks      | elapsed:   15.6s
[Parallel(n_jobs=16)]: Done 2594 tasks      | elapsed:   26.7s
[Parallel(n_jobs=16)]: Done 3260 out of 3260 | elapsed:   29.2s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.3s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    1.9s
[Parallel(n_jobs=16)]: Done 624 tasks      | elapsed:    6.6s
[Parallel(n_jobs=16)]: Done 968 tasks      | elapsed:    7.9s
[Parallel(n_jobs=16)]: Done 1424 tasks      | elapsed:   22.6s
[Parallel(n_jobs=16)]: Done 2344 tasks      | elapsed:   25.5s
[Parallel(n_jobs=16)]: Do

...

20180125


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.4s
[Parallel(n_jobs=16)]: Done 148 tasks      | elapsed:    1.9s
[Parallel(n_jobs=16)]: Done 576 tasks      | elapsed:    7.5s
[Parallel(n_jobs=16)]: Done 820 tasks      | elapsed:    8.6s
[Parallel(n_jobs=16)]: Done 1144 tasks      | elapsed:   22.7s
[Parallel(n_jobs=16)]: Done 1476 tasks      | elapsed:   24.6s
[Parallel(n_jobs=16)]: Done 2016 tasks      | elapsed:   26.9s
[Parallel(n_jobs=16)]: Done 2593 tasks      | elapsed:   37.5s
[Parallel(n_jobs=16)]: Done 3184 tasks      | elapsed:   40.0s
[Parallel(n_jobs=16)]: Done 3238 out of 3269 | elapsed:   40.2s remaining:    0.4s
[Parallel(n_jobs=16)]: Done 3269 out of 3269 | elapsed:   40.2s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.3s
[Parallel(n_jobs=16)]: Done 148 tasks      | elapsed:    1.8s
[Pa

20180126


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.3s
[Parallel(n_jobs=16)]: Done 148 tasks      | elapsed:    1.9s
[Parallel(n_jobs=16)]: Done 576 tasks      | elapsed:   10.7s
[Parallel(n_jobs=16)]: Done 872 tasks      | elapsed:   12.1s
[Parallel(n_jobs=16)]: Done 1496 tasks      | elapsed:   26.0s
[Parallel(n_jobs=16)]: Done 2344 tasks      | elapsed:   29.1s
[Parallel(n_jobs=16)]: Done 3120 tasks      | elapsed:   42.3s
[Parallel(n_jobs=16)]: Done 3265 out of 3265 | elapsed:   42.8s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    6.1s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    6.7s
[Parallel(n_jobs=16)]: Done 368 tasks      | elapsed:    7.7s
[Parallel(n_jobs=16)]: Done 692 tasks      | elapsed:    8.9s
Exception in thread QueueManagerThread:
Traceback (most recent call last):
  File "/usr/

KeyboardInterrupt: 

In [3]:

def get_period_stk_data(sid,start_date,end_date):
    try:
        stock_data = get_stk_bar(sid, freq="1m", start=start_date, end=end_date, fields=["open", "high", "low", "close"])
        stock_data = stock_data.reset_index().rename(columns={"index": "datetime"})
        stock_data["DataDate"]=stock_data["datetime"].apply(lambda x:str(x)[0:10].replace('-',''))
        df_pre_close = get_stk_bar(sid, freq="1d", start=start_date, end=end_date, fields=["pre_close"])
        df_pre_close = df_pre_close.reset_index().rename(columns={"date": "datetime"})
        df_pre_close["DataDate"]=df_pre_close["datetime"].apply(lambda x:str(x)[0:10].replace('-',''))
        df_pre_close = df_pre_close.drop(["datetime"],axis=1)
        stock_data = stock_data.merge(df_pre_close,on="DataDate")
        stock_data["sid"] = sid
    except Exception as e:
        print(e)
        return pd.DataFrame()
    return stock_data

def get_period_data(start_date, end_date):
    df=query_table("DailyBar",start_date=start_date,end_date=end_date,fields=["tradable"])
    df= df[df["tradable"]==1]
    stk_ls = df["sid"].drop_duplicates()

    results = Parallel(n_jobs=16, verbose=5, backend="loky", batch_size='auto') \
        (delayed(get_period_stk_data)(sid, start_date,end_date) for sid in stk_ls)
    data = pd.concat(results)

    return data.sort_values(["sid", "datetime"])

def append_yesterday_data(df):
    df["yesterday_close"] = df["close"].shift(240)
    return df.iloc[240:]

def data_store(df):
    date=df["DataDate"].iloc[0]
    df = df.drop(["DataDate"],axis=1)
    df.to_csv(r"/share/intern_share/stk_bundle_v2/crosscount_{}.csv".format(date), index_label="sid")
    return 0

def get_period_cross_data(start_date, end_date):
    data=get_period_data(start_date,end_date)
    
    data = data.groupby("sid",as_index = False).apply(append_yesterday_data)

    groups = data.groupby(["sid","DataDate"])
    cross_count_df = apply_parallel(groups, get_count_parallel)
    cross_count_df = cross_count_df.reset_index().drop(["sid"],axis=1).rename(columns={"level_0":"sid","level_1":"DataDate"})
    cross_count_df.groupby("DataDate").apply(data_store)
    return cross_count_df

get_period_cross_data('20180629','20180930')

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    2.9s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    6.0s
[Parallel(n_jobs=16)]: Done 256 tasks      | elapsed:   10.9s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   16.8s
[Parallel(n_jobs=16)]: Done 616 tasks      | elapsed:   24.2s
[Parallel(n_jobs=16)]: Done 850 tasks      | elapsed:   32.7s
[Parallel(n_jobs=16)]: Done 1120 tasks      | elapsed:   43.1s
[Parallel(n_jobs=16)]: Done 1426 tasks      | elapsed:   54.6s
[Parallel(n_jobs=16)]: Done 1768 tasks      | elapsed:  1.1min
[Parallel(n_jobs=16)]: Done 2146 tasks      | elapsed:  1.4min
[Parallel(n_jobs=16)]: Done 2560 tasks      | elapsed:  1.6min
[Parallel(n_jobs=16)]: Done 3010 tasks      | elapsed:  1.9min
[Parallel(n_jobs=16)]: Done 3495 out of 3495 | elapsed:  2.2min finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: 

,sid,DataDate,cross_count
0,000001.SZ,20180702,5
1,000001.SZ,20180703,2
2,000001.SZ,20180704,1
3,000001.SZ,20180705,2
4,000001.SZ,20180706,5
...,...,...,...
217926,603999.SH,20180921,1
217927,603999.SH,20180925,0
217928,603999.SH,20180926,0
217929,603999.SH,20180927,1


In [12]:
date_ls=get_trade_dates(20190102,20190102)
for date in date_ls:
    df=pd.read_csv(r"/share/intern_share/stk_bundle_v2/crosscount_{}.csv".format(date))
    df=df.drop(["sid"],axis=1).rename(columns={"sid.1":"sid"})
    df.to_csv(r"/share/intern_share/stk_bundle_v2/crosscount_{}.csv".format(date),index=False)

In [30]:
#written by wangyonghao

#the api: get_crosscount_summary(date,period_ls)

#input: date: current date ( "20190102" for example)
#       period_ls:the list for the window period ([5,10,15] for example)


#output to file: the excel file of the cross_grid_count_summary for each window_period and for each stock



import numpy as np
import pandas as pd
import tushare as ts
import click

def factor_data_calcu(td_date, stock_ls, window_period, factor_df, crosscount_df):
    date_idx = date_ls.index(td_date)
    date_range = date_ls[date_idx - window_period + 1:date_idx + 1]

    count = np.sum(np.array(crosscount_df.loc[date_range, stock_ls]), axis=0)

    factor_df.loc[stock_ls, "{}d_count".format(window_period)] = count

def get_crosscount_summary(date,period_ls):

    date_range = date_ls[max(0, date_ls.index(date) - 29):date_ls.index(date) + 1]
    crosscount_df = pd.DataFrame()
    for datadate in date_range:
        temp_df = pd.read_csv(r"/share/intern_share/stk_bundle_v2/crosscount_{}.csv".format(datadate), index_col="sid")
        temp_df = temp_df.reindex(pd.MultiIndex.from_product([[datadate], temp_df.index]), level=1)
        crosscount_df = pd.concat([crosscount_df, temp_df])
    crosscount_df = crosscount_df.unstack()["cross_count"]
    crosscount_df = crosscount_df.dropna(axis=1, how="any")

    factor_df = pd.DataFrame(index=crosscount_df.columns,
                             columns=[str(i)+"d_count" for i in period_ls])

    for window_period in period_ls:
        factor_data_calcu(date, crosscount_df.columns, window_period, factor_df, crosscount_df)
    factor_df["mean_5-mean_30"]=factor_df["5d_count"]/5-factor_df["30d_count"]/30
    factor_df["mean_10-mean_30"]=factor_df["10d_count"]/10-factor_df["30d_count"]/30

    factor_df.to_excel(r"/share/intern_share/stk_bundle_v2/crosscount_summary_{}.xlsx".format(date),
                       index_label="sid")
    
    stock_pool=pd.read_csv(r"/share/xfzhang/to_colleague/to_yzhao/task2/{}/{}/task2_{}.csv".\
                                        format(date[0:4],date[4:6],date[0:8]))["sid"]
    factor_df_baima=factor_df.loc[stock_pool]
    factor_df_baima.to_excel(r"/share/intern_share/stk_bundle_baima/crosscount_summary_{}.xlsx".format(date),
                       index_label="sid")

    print(date,"finished")


In [58]:
date_ls=trade_days+["20200102"]

get_crosscount_summary("20200102",[5,10,15,20,30])

20200102 finished


/home/ywang/env/dev/lib/python3.6/site-packages/ipykernel_launcher.py:50: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike


In [79]:
#统计 10d_ma_crosscount 大于10的股票

date_ls=get_trade_dates(20190301,20191029)

summary_df=pd.DataFrame()
for date in date_ls:
    temp_df=pd.read_excel(r"/share/intern_share/stk_bundle/crosscount_summary_{}.xlsx".format(date),index_col="sid")
    temp_df = temp_df.reindex(pd.MultiIndex.from_product([[date], temp_df.index]), level=1)
    for window in [5,10,15,20,30]:
        temp_df["{}d_ma>10".format(window)]=temp_df["{}d_count".format(window)]>10*window
    
    columns=["{}d_ma>10".format(i) for i in [5,10,15,20,30]]
    temp_df["any_ma>10"]=temp_df[columns].apply(lambda x:any(x),axis=1)
    summary_df=pd.concat([summary_df,temp_df])
    
summary_df=summary_df.reset_index().rename(columns={"level_0":"date";,"level_1":"sid"})

sid_date_df=summary_df[["sid","date"]][summary_df["any_ma>10"] == True]
sid_date_df.reset_index(drop=True,inplace=True)
sid_date_df.to_csv(r"data/crosscount_ma_>10_stocklist.csv",index=False)

sid_date_df.to_csv(r"/share/intern_share/assign_1030/crosscount_ma_>10_stocklist.csv",index=False)